In [2]:
import gym
import numpy as np
env = gym.make('CartPole-v0')

[2016-11-28 20:46:37,863] Making new env: CartPole-v0


In [5]:
env.reset()
random_episodes = 0
reward_sum = 0
while random_episodes < 10:
    env.render()
    observation, reward, done, _ = env.step(np.random.randint(0,2))
    print("observation = ", observation)
    print ("reward = ", reward)
    print ("done = ", done)
    print ("random_episodes = ", random_episodes)
    reward_sum += reward
    print ("##### reward_sum = ", reward_sum)
    
    if done:
        random_episodes += 1
        print "Reward for this episode was:",reward_sum
        reward_sum = 0
        env.reset()

('observation = ', array([-0.00822438,  0.21070934,  0.01277674, -0.28258781]))
('reward = ', 1.0)
('done = ', False)
('random_episodes = ', 0)
('##### reward_sum = ', 1.0)
('observation = ', array([-0.00401019,  0.40564674,  0.00712499, -0.57121379]))
('reward = ', 1.0)
('done = ', False)
('random_episodes = ', 0)
('##### reward_sum = ', 2.0)
('observation = ', array([ 0.00410274,  0.60066806, -0.00429929, -0.86164361]))
('reward = ', 1.0)
('done = ', False)
('random_episodes = ', 0)
('##### reward_sum = ', 3.0)
('observation = ', array([ 0.01611611,  0.79584829, -0.02153216, -1.15567523]))
('reward = ', 1.0)
('done = ', False)
('random_episodes = ', 0)
('##### reward_sum = ', 4.0)
('observation = ', array([ 0.03203307,  0.99124427, -0.04464567, -1.45503122]))
('reward = ', 1.0)
('done = ', False)
('random_episodes = ', 0)
('##### reward_sum = ', 5.0)
('observation = ', array([ 0.05185796,  1.18688495, -0.07374629, -1.7613213 ]))
('reward = ', 1.0)
('done = ', False)
('random_episodes

In [11]:
import tensorflow as tf
# hyperparameters
H = 10 # number of hidden layer neurons
batch_size = 50 # every how many episodes to do a param update?
learning_rate = 1e-2 # feel free to play with this to train faster or more stably.
gamma = 0.99 # discount factor for reward

D = 4 # input dimensionality

In [12]:
tf.reset_default_graph()

#This defines the network as it goes from taking an observation of the environment to 
#giving a probability of chosing to the action of moving left or right.
observations = tf.placeholder(tf.float32, [None,D] , name="input_x")

W1 = tf.get_variable("W1", shape=[D, H],
           initializer=tf.contrib.layers.xavier_initializer())

layer1 = tf.nn.relu(tf.matmul(observations,W1))

W2 = tf.get_variable("W2", shape=[H, 1],
           initializer=tf.contrib.layers.xavier_initializer())

score = tf.matmul(layer1,W2)
print ("score = ", score)

probability = tf.nn.sigmoid(score)
print ("probability = ", probability)

#From here we define the parts of the network needed for learning a good policy.
tvars = tf.trainable_variables()
input_y = tf.placeholder(tf.float32,[None,1], name="input_y")
advantages = tf.placeholder(tf.float32,name="reward_signal")

# The loss function. This sends the weights in the direction of making actions 
# that gave good advantage (reward over time) more likely, and actions that didn't less likely.
loglik = tf.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
loss = -tf.reduce_mean(loglik * advantages) 
newGrads = tf.gradients(loss,tvars)

# Once we have collected a series of gradients from multiple episodes, we apply them.
# We don't just apply gradeients after every episode in order to account for noise in the reward signal.
adam = tf.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer
W1Grad = tf.placeholder(tf.float32,name="batch_grad1") # Placeholders to send the final gradients through when we update.
W2Grad = tf.placeholder(tf.float32,name="batch_grad2")
batchGrad = [W1Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))

('score = ', <tf.Tensor 'MatMul_1:0' shape=(?, 1) dtype=float32>)
('probability = ', <tf.Tensor 'Sigmoid:0' shape=(?, 1) dtype=float32>)


In [13]:

def discount_rewards(r):
    """ take 1D float array of rewards and compute discounted reward """
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [14]:
xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]

running_reward = None
reward_sum = 0
episode_number = 1
total_episodes = 10000
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    rendering = False
    sess.run(init)
    observation = env.reset() # Obtain an initial observation of the environment

    # Reset the gradient placeholder. We will collect gradients in 
    # gradBuffer until we are ready to update our policy network. 
    gradBuffer = sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while episode_number <= total_episodes:
        
        # Rendering the environment slows things down, 
        # so let's only look at it once our agent is doing a good job.
        if reward_sum/batch_size > 100 or rendering == True : 
            env.render()
            rendering = True
            
        # Make sure the observation is in a shape the network can handle.
        x = np.reshape(observation,[1,D])
        
        # Run the policy network and get an action to take. 
        tfprob = sess.run(probability,feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        xs.append(x) # observation
        y = 1 if action == 0 else 0 # a "fake label"
        ys.append(y)

        # step the environment and get new measurements
        observation, reward, done, info = env.step(action)
        reward_sum += reward

        drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)

        if done: 
            episode_number += 1
            # stack together all inputs, hidden states, action gradients, and rewards for this episode
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            tfp = tfps
            xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[] # reset array memory

            # compute the discounted reward backwards through time
            discounted_epr = discount_rewards(epr)
            # size the rewards to be unit normal (helps control the gradient estimator variance)
            discounted_epr -= np.mean(discounted_epr)
            discounted_epr /= np.std(discounted_epr)
            
            # Get the gradient for this episode, and save it in the gradBuffer
            tGrad = sess.run(newGrads,feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
                
            # If we have completed enough episodes, then update the policy network with our gradients.
            if episode_number % batch_size == 0: 
                sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W2Grad:gradBuffer[1]})
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
                
                # Give a summary of how well our network is doing for each batch of episodes.
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                print 'Average reward for episode %f.  Total average reward %f.' % (reward_sum/batch_size, running_reward/batch_size)
                
                if reward_sum/batch_size > 200: 
                    print "Task solved in",episode_number,'episodes!'
                    break
                    
                reward_sum = 0
            
            observation = env.reset()
        
print episode_number,'Episodes completed.'

Average reward for episode 17.960000.  Total average reward 17.960000.
Average reward for episode 21.760000.  Total average reward 17.998000.
Average reward for episode 22.200000.  Total average reward 18.040020.
Average reward for episode 21.640000.  Total average reward 18.076020.
Average reward for episode 24.200000.  Total average reward 18.137260.
Average reward for episode 21.820000.  Total average reward 18.174087.
Average reward for episode 24.280000.  Total average reward 18.235146.
Average reward for episode 22.860000.  Total average reward 18.281395.
Average reward for episode 26.760000.  Total average reward 18.366181.
Average reward for episode 25.040000.  Total average reward 18.432919.
Average reward for episode 24.400000.  Total average reward 18.492590.
Average reward for episode 27.180000.  Total average reward 18.579464.
Average reward for episode 30.880000.  Total average reward 18.702469.
Average reward for episode 29.240000.  Total average reward 18.807845.
Averag